# Visualizações interativas - Bokeh

Este notebook apresenta os conceitos básicos de visualizações interativas usando a biblioteca [Bokeh](https://bokeh.pydata.org).

Exemplos baseados no tutorial [Data Visualization with Bokeh in Python](https://towardsdatascience.com/data-visualization-with-bokeh-in-python-part-one-getting-started-a11655a467d4).

## Leitura e análise inicial dos dados


In [1]:
#importing libraries
import pandas as pd

%matplotlib inline

# lê o arquivo CSV
df = pd.read_csv('../data/aluguel.csv')
df = df.set_index('codigo')
df['data'] = pd.to_datetime(df['data'], format='%d/%m/%y')

Vamos utilizar um dataset de ofertas de aluguel. O primeiro passo ao se analisar dados desconhecidos é visualizar algumas linhas de dados:

In [2]:
df.head(10)

,endereco,quartos,suite,area,vaga,aluguel,condominio,data
codigo,,,,,,,,
34,Rua Desembargador Westphalen,2,0,90,0,900,371,2017-10-11
167,Rua Jose Loureiro,2,0,64,0,650,428,2017-07-15
6784,Rua Jose Loureiro,2,0,81,0,1100,400,2017-08-23
82,Rua Lourenço Pinto,2,0,50,0,1350,300,2017-09-19
2970,Rua Lourenço Pinto,2,0,63,0,1300,300,2017-08-05
34197,Alameda Doutor Muricy,2,0,80,1,900,410,2017-10-23
5072,Alameda Doutor Muricy,2,0,84,0,1100,382,2017-09-02
469,Rua Desembargador Westphalen,1,0,30,0,550,210,2017-07-03
24,Rua Desembargador Westphalen,1,0,60,1,800,120,2017-09-30


Vamos usar em alguns exemplos um DataFrame com os valores de condomínio e aluguel agregados por mês da oferta:

In [3]:
df_mensal = df.groupby(df.data.dt.to_period("M"))[['aluguel', 'condominio']].mean()
df_mensal

,aluguel,condominio
data,,
2017-07,670.0,313.8
2017-08,1000.0,346.0
2017-09,942.0,269.4
2017-10,980.0,414.2


## Visualização com o Bokeh

Na biblioteca Bokeh, gráficos são construídos em camadas. O primeiro passo é criar uma figura e então adicionar as camadas de elementos (chamados *glyphs*).

Glyphs podem ser usados para plotar círculos, retângulos, arcos, etc. O exemplo abaixo cria uma figura e adiciona dois glyphs: um com quadrados para representar apartamentos de 1 quarto e um com círculos para representar os de 2 quartos.

In [4]:
# configura para gerar o gráfico diretamente no notebook
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
output_notebook()

# Criação da figura e configuração dos rótulos
p = figure(plot_width = 600, plot_height = 600, 
           title = 'Ofertas de Aluguel',
           x_axis_label = 'Área', y_axis_label = 'Aluguel')

# Dados a plotar
df_1q = df[df['quartos'] == 1]
df_2q = df[df['quartos'] == 2]


# adiciona glyph com quadrados
p.square(df_1q['area'], df_1q['aluguel'], size = 12, color = 'navy', alpha = 0.6)
# adiciona glyph com círculos
p.circle(df_2q['area'], df_2q['aluguel'], size = 12, color = 'red', alpha = 0.6)

# Exibe a figura
show(p)

Loading BokehJS ...

Podemos alterar o tamanho dos elementos passando uma lista (ou Series) com os tamanhos de cada elemento como no exemplo abaixo. Perceba também que já é possível interagir com o gráfico usando a barra de ferramentas à direita.

In [5]:
p = figure(plot_width = 600, plot_height = 600, 
           title = 'Ofertas de Aluguel',
           x_axis_label = 'Área', y_axis_label = 'Aluguel')

df_1q = df[df['quartos'] == 1]
df_2q = df[df['quartos'] == 2]


p.circle(df_1q['area'], df_1q['aluguel'], size = df_1q['condominio']/10, color = 'navy', alpha = 0.6)

p.circle(df_2q['area'], df_2q['aluguel'], size = df_2q['condominio']/10, color = 'red', alpha = 0.6)

show(p)

Um gráfico de linhas pode ser criado de forma similar:

In [6]:
p = figure(title="Evolução das médias de aluguel e condomínio", 
           x_axis_type="datetime", x_axis_label='Data', y_axis_label='Valor')

# Adicionando glyph de linha para uma variável
p.line('data', 'aluguel', source = df_mensal, legend="Aluguel", line_width=2)

p.line('data', 'condominio', source = df_mensal, color = 'red', legend="Condomínio", line_width=2)

show(p)

## Interatividade com os elementos

O tipo mais básico de interação permitida pelo Bokeh é a interação passiva, na qual interagimos com os objetos mas não alteramos o conjunto de dados sendo exibidos. Este tipo de visualização é muito útil e simples de se implementar.

Neste exemplo vamos adicionar informações sobre os pontos exibidos no gráfico quando o usuário passar o mouse sobre os elementos. Para isto precisamos converter nossa DataFrame em uma estrutura de dados similar utilizada pelo Bokeh chamada ColumnDataSource:

In [7]:
# Importando a classe ColumnDataSource
from bokeh.models import ColumnDataSource

# convertendo o DataFrame
src = ColumnDataSource(df)

# exibindo as chaves e um valor armazenado na nova estrutura
print(src.data.keys())
print(src.data['endereco'][1])

dict_keys(['codigo', 'endereco', 'quartos', 'suite', 'area', 'vaga', 'aluguel', 'condominio', 'data'])
Rua Jose Loureiro


Para exibir informações quando o mouse passa sobre os elementos, usaremos a ferramenta HoverTool. Ao definir a ferramenta, adicionamos as informações desejadas informando as colunas que as contêm (usando @ antes do nome da coluna). Use o mouse para ver o resultado.

In [8]:
from bokeh.models import HoverTool

p = figure(plot_width = 600, plot_height = 600, 
           title = 'Ofertas de Aluguel',
           x_axis_label = 'Área', y_axis_label = 'Aluguel')

# Adiciona círculos glyph, passando a ColumnDataSource como fonte de dados
p.circle('area', 'aluguel', source = src, size = 20, color = 'navy', hover_fill_color = 'red', alpha = 0.6, hover_fill_alpha = 1.0)

# Adiciona texto que é mostrado quando se passa o mouse
hover = HoverTool(tooltips = [('Condomínio', '@condominio'),
                             ('Num quartos', '@quartos')])

# Adiciona a funcionalidade de hover tool à figura
p.add_tools(hover)

show(p)

## Salvando o gráfico em um arquivo html

In [9]:
## Import savings function

from bokeh.io import output_file
# Para salvar um html com o gráfico, descomente e execute a linha abaixo
# output_file('aluguel.html')
# show(p)

### Referências

- Documentção do Bokeh: [Quickstart](https://bokeh.pydata.org/en/latest/docs/user_guide/quickstart.html)